In [3]:
#IMPORTO LAS LIBRERIAS E IMPORTO EL CSV
#######################################

import numpy as np
import pandas as pd

data = pd.read_csv("C:/Users/abidart/Desktop/Curso/properatti.csv")

In [4]:
#PASO EL CSV A DATAFRAME
########################

data = pd.DataFrame(data)


In [6]:
#RENOMBRO LA PRIMERA COLUMNA
############################

data.rename(columns={'Unnamed: 0':'index'}, inplace=True)
data.head()

In [33]:
#ANALIZO PRINCIPALES ESTADISTICOS
#################################

data.describe()

,index,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,121220.000000,1.025030e+05,69670.000000,69670.000000,1.008100e+05,1.008100e+05,1.008100e+05,81892.000000,101313.000000,68617.000000,8.765800e+04,7899.000000,47390.000000,1.426200e+04
mean,60609.500000,3.574442e+06,-34.626210,-59.266290,4.685259e+05,4.229397e+06,2.397006e+05,233.795328,133.050181,2160.086916,6.912216e+03,17.452336,3.080840,5.009234e+03
std,34993.344153,3.541306e+05,1.980936,2.299922,2.260101e+06,6.904714e+06,3.913239e+05,1782.222147,724.351479,2759.288621,2.837864e+04,120.243621,1.860773,1.204403e+05
min,0.000000,3.427208e+06,-54.823985,-75.678931,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.600000,1.510204e+00,1.000000,1.000000,1.000000e+00
25%,30304.750000,3.430234e+06,-34.669065,-58.727040,1.100000e+05,1.583309e+06,8.973388e+04,50.000000,45.000000,1218.181818,1.550000e+03,1.000000,2.000000,1.000000e+03
50%,60609.500000,3.433910e+06,-34.597985,-58.480128,1.850000e+05,2.558452e+06,1.450000e+05,84.000000,75.000000,1800.000000,2.213115e+03,3.000000,3.000000,2.000000e+03
75%,90914.250000,3.836668e+06,-34.441299,-58.395908,4.200000e+05,4.675792e+06,2.650000e+05,200.000000,150.000000,2486.411765,3.355549e+03,6.000000,4.000000,4.000000e+03
max,121219.000000,6.948895e+06,4.545843,-53.733330,6.500000e+08,8.212711e+08,4.654544e+07,200000.000000,187000.000000,206333.333333,4.000000e+06,3150.000000,32.000000,1.000150e+07


In [35]:
#CHEQUEO CANTIDAD DE NULOS
#################################

data.isnull().sum()

index                              0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64

In [8]:
#ANALIZO CORRELACION ENTRE VARIABLES
####################################

datacorr = data.corr(method='pearson')
datacorr.style.background_gradient(cmap='coolwarm')


In [43]:
#IMPUTO LOS M2 TOTALES CUANDO TENGO LOS CUBIERTOS EN BASE A UNA RATIO DE LA MEDIA ENTRE TOTALES / CUBIERTOS
###########################################################################################################

data['Ratio_m2']=data['surface_covered_in_m2']/data['surface_total_in_m2']  
data['Media_Ratio_m2']= data['Ratio_m2'].mean()
data['surface_total_in_m2_Ratio'] = np.where((data['surface_total_in_m2'].isna()) & (data['surface_covered_in_m2'].notnull()) ,  data['surface_covered_in_m2'] / data['Media_Ratio_m2'] , data['surface_total_in_m2'])


#CHEQUEO UN CASO PARTICULAR
###########################

data.loc[data['index'] == 68]

In [42]:
#SPLIT SOBRE LA COLUMNA CON LA GEOGRAFIA PARA OBTENER CUIDAD Y BARRIOS
######################################################################

new = data['place_with_parent_names'].str.split("|", expand = True) 
data["Country"]= new[1] 
data["City"]= new[2]
data["Neighborhood"]= new[3] 
data["Zone 1"]= new[4] 
data["Zone 2"]= new[5] 
data["Zone 3"]= new[6] 


#PROMEDIO DE M2 POR BARRIO USANDO VARIABLE DE M2 CON RATIO
##########################################################

data['price_per_m2_Ratio']=data['price_aprox_usd']/data['surface_total_in_m2_Ratio'] 
data.groupby('Neighborhood')['price_per_m2_Ratio'].mean()

